In [92]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import music21

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [93]:
x = np.load('input.npy')
y = np.load('output.npy')

input_length = x.shape[1]
print(input_length)
output_length = y.shape[1]
print(output_length)

3328
9856


In [94]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(input_length,)),    
    keras.layers.Dense(200, activation='relu'),    
    keras.layers.Dense(output_length, activation='sigmoid')
])
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_11 (Flatten)        (None, 3328)              0         
                                                                 
 dense_22 (Dense)            (None, 200)               665800    
                                                                 
 dense_23 (Dense)            (None, 9856)              1981056   
                                                                 
Total params: 2,646,856
Trainable params: 2,646,856
Non-trainable params: 0
_________________________________________________________________


In [95]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])

In [96]:
model.fit(x, y, epochs=50)
model.save('trained_model.h5')

Epoch 1/50
139/139 [==============================] - 5s 31ms/step - loss: 0.0723 - binary_accuracy: 0.9853
Epoch 2/50
139/139 [==============================] - 4s 32ms/step - loss: 0.0253 - binary_accuracy: 0.9930
Epoch 3/50
139/139 [==============================] - 4s 31ms/step - loss: 0.0247 - binary_accuracy: 0.9932
Epoch 4/50
139/139 [==============================] - 4s 32ms/step - loss: 0.0240 - binary_accuracy: 0.9934
Epoch 5/50
139/139 [==============================] - 4s 32ms/step - loss: 0.0231 - binary_accuracy: 0.9935
Epoch 6/50
139/139 [==============================] - 4s 32ms/step - loss: 0.0226 - binary_accuracy: 0.9935
Epoch 7/50
139/139 [==============================] - 5s 33ms/step - loss: 0.0223 - binary_accuracy: 0.9936
Epoch 8/50
139/139 [==============================] - 4s 32ms/step - loss: 0.0221 - binary_accuracy: 0.9936
Epoch 9/50
139/139 [==============================] - 4s 32ms/step - loss: 0.0219 - binary_accuracy: 0.9936
Epoch 10/50
139/139 [=======

Jetzt unterteilt man die Predictions in die verschieden Stimmen mit folgender Rechnung. Wir wissen, dass wir eine Tonrange bei Alto von 23+1 haben. Wir wissen auch unsere Chunckgröße von 128. Also:

128*24=3072

182*23=2944
3072+2944=6016

...

In [97]:
soprano = x[200]
predictions = model.predict(soprano.reshape(1, -1))

predictions = predictions.reshape(-1)

soprano = soprano.reshape(128, -1)
alto = predictions[:3072].reshape(128, -1)
tenor = predictions[3072:6016].reshape(128, -1)
bass = predictions[6016:9856].reshape(128, -1)

music = {
    'soprano': soprano,
    'alto': alto,
    'tenor': tenor,
    'bass': bass
}

1/1 [==============================] - 0s 46ms/step


In [98]:
SOPRANO_MIN = 57
SOPRANO_MAX = 81

ALTO_MIN = 52
ALTO_MAX = 74

TENOR_MIN = 48
TENOR_MAX = 69

BASS_MIN = 36
BASS_MAX = 64

ranges = {
    'soprano': {midinumber: (midinumber - SOPRANO_MIN + 1) for midinumber in range(SOPRANO_MIN, SOPRANO_MAX + 1)},
    'alto': {midinumber: (midinumber - ALTO_MIN + 1) for midinumber in range(ALTO_MIN, ALTO_MAX + 1)},
    'tenor': {midinumber: (midinumber - TENOR_MIN + 1) for midinumber in range(TENOR_MIN, TENOR_MAX + 1)},
    'bass': {midinumber: (midinumber - BASS_MIN + 1) for midinumber in range(BASS_MIN, BASS_MAX + 1)},
}

reverse_ranges = {
    'soprano': {(midinumber - SOPRANO_MIN + 1): midinumber for midinumber in range(SOPRANO_MIN, SOPRANO_MAX + 1)},
    'alto': {(midinumber - ALTO_MIN + 1): midinumber for midinumber in range(ALTO_MIN, ALTO_MAX + 1)},
    'tenor': {(midinumber - TENOR_MIN + 1): midinumber for midinumber in range(TENOR_MIN, TENOR_MAX + 1)},
    'bass': {(midinumber - BASS_MIN + 1): midinumber for midinumber in range(BASS_MIN, BASS_MAX + 1)},
}

In [99]:
def decode_note(n, rang):
    if n == 0:
        ret = '--'
    else:
        note = music21.note.Note(type='16th')        
        note.pitch.midi = reverse_ranges[rang][n]        
        ret = note
    return ret

In [100]:
generation = {
    'soprano': [],
    'alto': [],
    'tenor': [],
    'bass': []
}

for sixteenth in range(128):
    for part, notes in music.items():
        this_note = decode_note(np.argmax(notes[sixteenth]), part)
        if this_note == '--':
            last_note = generation[part][-1]
            this_note = music21.note.Note(last_note.pitch.nameWithOctave, type='16th')
            if last_note.tie:
                this_note.tie = music21.tie.Tie('continue')
            else:
                last_note.tie = music21.tie.Tie('start')
                generation[part][-1] = last_note
                this_note.tie = music21.tie.Tie('continue')
        else:
            if sixteenth > 0:
                last_note = generation[part][-1]
                if last_note.tie:
                    last_note.tie = music21.tie.Tie('stop')
        generation[part].append(this_note)

In [101]:
df = pd.DataFrame(generation)
print(df)

                  soprano                   alto                  tenor  \
0   <music21.note.Note B>  <music21.note.Note G>  <music21.note.Note D>   
1   <music21.note.Note B>  <music21.note.Note G>  <music21.note.Note D>   
2   <music21.note.Note B>  <music21.note.Note G>  <music21.note.Note D>   
3   <music21.note.Note B>  <music21.note.Note G>  <music21.note.Note D>   
4   <music21.note.Note B>  <music21.note.Note G>  <music21.note.Note D>   
5   <music21.note.Note B>  <music21.note.Note G>  <music21.note.Note D>   
6   <music21.note.Note B>  <music21.note.Note G>  <music21.note.Note D>   
7   <music21.note.Note B>  <music21.note.Note G>  <music21.note.Note D>   
8   <music21.note.Note A>  <music21.note.Note F>  <music21.note.Note C>   
9   <music21.note.Note A>  <music21.note.Note F>  <music21.note.Note C>   
10  <music21.note.Note A>  <music21.note.Note F>  <music21.note.Note C>   
11  <music21.note.Note A>  <music21.note.Note F>  <music21.note.Note C>   
12  <music21.note.Note A>

In [102]:
s = music21.stream.Stream()
s.append(df.soprano.to_list())
a = music21.stream.Stream()
a.append(df.alto.to_list())
t = music21.stream.Stream()
t.append(df.tenor.to_list())
b = music21.stream.Stream()
b.append(df.bass.to_list())
stream = music21.stream.Stream([s,a,t,b])

In [103]:
stream.write('musicxml', 'example.musicxml')

WindowsPath('d:/Projekte/Python/Tensorflow/Komposition-eines-Musikstuecks-mittels-Neuronaler-Netze/mukkeBude/example.musicxml')